In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

In [ ]:
train_data = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
train_data = train_data.fillna(method = 'bfill')
train_data.info()

In [ ]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)
train_x, train_y = make_train_data(train_data, 5, 200)

In [ ]:
train_x = np.array(train_x).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
train_y = np.array(train_y)

train_x.shape, train_y.shape

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

tf.keras.utils.set_random_seed(RANDOM_STATE)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
model.fit(train_x, train_y, epochs=10, batch_size=128);

2022-07-11 05:43:39.023468: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823553280 exceeds 10% of free system memory.
2022-07-11 05:43:39.390627: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823553280 exceeds 10% of free system memory.


Epoch 1/10


2022-07-11 05:43:41.122092: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  3/204 [..............................] - ETA: 6s - loss: 294314.1250 - mae: 345.3499  

2022-07-11 05:43:41.655113: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


204/204 [==============================] - 10s 37ms/step - loss: 190296.2344 - mae: 332.8742
Epoch 2/10
204/204 [==============================] - 8s 37ms/step - loss: 165485.1719 - mae: 317.3720
Epoch 3/10
204/204 [==============================] - 8s 37ms/step - loss: 155167.8438 - mae: 304.7706
Epoch 4/10
204/204 [==============================] - 7s 37ms/step - loss: 147070.8125 - mae: 294.5350
Epoch 5/10
204/204 [==============================] - 8s 37ms/step - loss: 140878.0156 - mae: 285.9803
Epoch 6/10
204/204 [==============================] - 8s 37ms/step - loss: 135573.2031 - mae: 278.6196
Epoch 7/10
204/204 [==============================] - 8s 37ms/step - loss: 130233.1250 - mae: 271.8609
Epoch 8/10
204/204 [==============================] - 7s 37ms/step - loss: 125357.4844 - mae: 266.0392
Epoch 9/10
204/204 [==============================] - 8s 37ms/step - loss: 120646.9219 - mae: 260.1277
Epoch 10/10
204/204 [==============================] - 8s 37ms/step - loss: 117090.

In [8]:
test_data_list = [x for x in range(196, 201)]
test_data      = train_data[train_data["Day"].isin(test_data_list)]
test_data      = test_data.drop(["TurbID", "Day"], axis=1)
test_data      = np.array(test_data).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
test_data.shape

(134, 720, 11)

In [9]:
sample_submission['Patv'] = model.predict(test_data).reshape(-1)
sample_submission.to_csv(join(PATH.output, "baseline1.csv"), index=False)

5/5 [==============================] - 0s 13ms/step
